In [1]:
import keras
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D,GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import numpy.random as rng
import numpy as np
import os
import cv2
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = rng.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)
#//TODO: figure out how to initialize layer biases in keras.
def b_init(shape,name=None):
    """Initialize bias as in paper"""
    values=rng.normal(loc=0.5,scale=1e-2,size=shape)
    return K.variable(values,name=name)

In [3]:
input_shape = (105, 105, 3)
left_input = Input(input_shape)
right_input = Input(input_shape)

In [4]:
vgg16_model = VGG16(weights = 'imagenet', include_top = False)
x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init)(x)
convnet=Model(input = vgg16_model.input, output = predictions)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """


In [5]:
convnet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [6]:
#call the convnet Sequential model on each of the input tensors so params will be shared
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#layer to merge two encoded inputs with the l1 distance between them
L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
#call this layer on list of two input tensors.
L1_distance = L1_layer([encoded_l, encoded_r])
prediction = Dense(1,activation='sigmoid',bias_initializer=b_init)(L1_distance)
siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(0.00006)
#//TODO: get layerwise learning rates and momentum annealing scheme described in paperworking
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()

16820033

In [7]:
siamese_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 4096)         16815936    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           model_1[1][0]                    
          

In [8]:
# freeze all layers of the pre-trained model
for layer in vgg16_model.layers:
    layer.trainable = False

In [9]:
siamese_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 105, 105, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 4096)         16815936    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 4096)         0           model_1[1][0]                    
          

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [10]:
PATH = "./Bongard/BP_61" #CHANGE THIS - path where the pickled data is stored

with open(os.path.join(PATH, "train.pickle"), "rb") as f:
    (X,c) = pickle.load(f)

with open(os.path.join(PATH, "val.pickle"), "rb") as f:
    (Xval,cval) = pickle.load(f)
    
print("training alphabets")
print(c.keys())
print("validation alphabets:")
print(cval.keys())

training alphabets
dict_keys(['right', 'left'])
validation alphabets:
dict_keys(['right', 'left'])


In [11]:
class Siamese_Loader:
    """For loading batches and testing tasks to a siamese net"""
    def __init__(self, path, data_subsets = ["train", "val"]):
        self.data = {}
        self.categories = {}
        self.info = {}
        
        for name in data_subsets:
            file_path = os.path.join(path, name + ".pickle")
            print("loading data from {}".format(file_path))
            with open(file_path,"rb") as f:
                (X,c) = pickle.load(f)
                self.data[name] = X
                self.categories[name] = c

    def get_batch(self,batch_size,s="train"):
        """Create batch of n pairs, half same class, half different class"""
        X=self.data[s]
        n_classes, n_examples, w, h = X.shape

        #randomly sample several classes to use in the batch
        categories = rng.choice(n_classes,size=(batch_size,),replace=False)
        #initialize 2 empty arrays for the input image batch
        #pairs=[np.zeros((batch_size, h, w,1)) for i in range(2)]
        pairs=[np.zeros((batch_size, h, w,3)) for i in range(2)]
        #initialize vector for the targets, and make one half of it '1's, so 2nd half of batch has same class
        targets=np.zeros((batch_size,))
        targets[batch_size//2:] = 1
        for i in range(batch_size):
            category = categories[i]
            idx_1 = rng.randint(0, n_examples)
            #pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 1)
            
            pairs[0][i,:,:,:] = cv2.cvtColor(cv2.resize(X[category, idx_1], (w, h)), cv2.COLOR_GRAY2RGB)
            idx_2 = rng.randint(0, n_examples)
            #pick images of same class for 1st half, different for 2nd
            if i >= batch_size // 2:
                category_2 = category  
            else: 
                #add a random number to the category modulo n classes to ensure 2nd image has
                # ..different category
                category_2 = (category + rng.randint(1,n_classes)) % n_classes
            #pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
            pairs[1][i,:,:,:]=cv2.cvtColor(cv2.resize(X[category_2,idx_2], (w, h)), cv2.COLOR_GRAY2RGB)
        return pairs, targets
    
    def generate(self, batch_size, s="train"):
        """a generator for batches, so model.fit_generator can be used. """
        while True:
            pairs, targets = self.get_batch(batch_size,s)
            yield (pairs, targets)   
            
    def test_bongard(self,model,N,k,s='val',s1='train',verbose=0):
        
        X_train=self.data[s1]
        X_val=self.data[s]
        
        n_classes, n_examples, w, h = X_train.shape
        m = n_classes*n_examples
        X_train=X_train.reshape(m, w, h,1)
        X_train = [cv2.cvtColor(cv2.resize(i, (w, h)), cv2.COLOR_GRAY2RGB) for i in X_train]
        X_train=np.array(X_train)

        
        n_classes_val, n_examples_val, w, h = X_val.shape
        m_val = n_classes_val * n_examples_val
        X_val=X_val.reshape(m_val, w, h,1)
        X_val = [cv2.cvtColor(cv2.resize(i, (w, h)), cv2.COLOR_GRAY2RGB) for i in X_val]
        X_val=np.array(X_val)
        
        
        n_correct = 0
        
        for i in range(k):
            
            test_image = np.asarray([X_val[i,:,:,:]]*m).reshape(m, w, h,3)
            support_set =  X_train
            if i < k/2:
                targets=0
            else:
                targets=1 
            
            pairs = [test_image,support_set]
            
            probs = model.predict(pairs)
            if np.argmax(probs)<6:
                test_result=0
            else:
                test_result=1
            if test_result== targets:
                n_correct+=1
                
        percent_correct = (100.0*n_correct / k)
        if verbose:
            print("Got an average of {}% {} way one-shot learning accuracy".format(percent_correct,N))
        return percent_correct
            
    def train(self, model, epochs, verbosity):
        model.fit_generator(self.generate(batch_size),
                            
                             )
    
    
#Instantiate the class
loader = Siamese_Loader(PATH)

loading data from ./Bongard/BP_61/train.pickle
loading data from ./Bongard/BP_61/val.pickle


In [ ]:
#Training loop
print("!")
evaluate_every = 1 # interval for evaluating on one-shot tasks
loss_every=50 # interval for printing loss (iterations)
batch_size = 2
n_iter = 200
N_way = 2 # how many classes for testing one-shot tasks>
n_val = 1000 #how mahy one-shot tasks to validate on?
best = 0.9
weights_path = os.path.join(PATH, "weights")
print("training")
for i in range(1, n_iter):
    (inputs,targets)=loader.get_batch(batch_size)
    loss=siamese_net.train_on_batch(inputs,targets)
    print(loss)
    if i > n_iter/2:
        print("evaluating")
        val_acc = loader.test_bongard(siamese_net,N_way,n_val,s='val',s1='train',verbose=True)
        if val_acc >= best:
            print("saving")
            siamese_net.save(weights_path)
            best=val_acc

    if i % loss_every == 0:
        print("iteration {}, training loss: {:.2f},".format(i,loss))

!
training


/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0.95217305
0.8802153
0.9425515
0.8230851
0.7964079
0.88215125
0.7999219
1.0337662
0.639658
0.60193086
1.1992917
1.0023569
0.91177136
1.0685813
0.8904298
1.0115752
0.83753735
0.98123217
0.8482621
0.84103376
0.95719457
1.02994
0.96457887
0.93035066
0.80721533
1.0187571
0.86566377
1.0011265
0.89179754
0.8713036
0.8522986
0.98920023
0.8146583
0.73497164
0.8704916
0.9358196
0.6520942
0.64870787
0.90088105
0.7417245
0.55683106
1.288465
0.5711302
0.49326605
1.2889993
0.94445956
0.8574864
0.94398504
1.0775361
0.98389125
iteration 50, training loss: 0.98,
0.8932611
0.8824159
0.891732
0.9240705
0.8136416
0.88514715
0.81870615
0.81858313
0.92680913
0.7480073
0.935487
0.7215479
0.84435326
0.84233356
0.8613289
0.77955526
0.7826294
0.86713094
0.7525333
1.0460972
0.7719383
0.6912103
0.7907069
0.80094755
0.91547805
0.6857964
1.0336889
0.86798626
0.8410957
0.8686414
0.83488137
0.8206063
0.7978583
0.79293996
0.7868351
0.92702115
0.94301176
0.8474517
0.8468558
0.89966863
0.82711667
0.93254554
0.6801464
0

In [ ]:
#print("evaluating")
#val_acc = loader.test_bongard(siamese_net,N_way,n_val,s='val',s1='train',verbose=True)
#if val_acc >= best:
    #print("saving")
    #siamese_net.save(weights_path)
    #best=val_acc